In [1]:
from edgar_functions import *
from name_mappings import (
    balance_sheet_mapping,
    income_statement_mapping,
    cash_flow_mapping,
)


tickers = [
    "AAPL",
    "MSFT",
    "AMZN",
    "NVDA",
    "GOOGL",
    "TSLA",
    "GOOG",
    "BRK-A",
    "META",
    "UNH",
    "XOM",
    "LLY",
    "JNJ",
    "V",
    "PG",
    "MA",
    "HD",
    "CVX",
    "MRK",
    "ABBV",
    "COST",
    "PEP",
    "WSM",
    "ADBE",
    "LRCX",
]
ticker = tickers[18]
ticker

'MRK'

In [2]:
print_links_to_desired_statment(ticker, "income_statement")

https://www.sec.gov/Archives/edgar/data/0000310158/000162828023005061/R3.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015822000003/R3.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015821000004/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015820000005/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015819000014/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015818000005/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015817000010/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015816000063/R2.htm
https://www.sec.gov/Archives/edgar/data/0000310158/000031015815000005/R2.htm


In [3]:
# print_links_to_desired_statment(ticker, "balance_sheet")

In [4]:
# print_links_to_desired_statment(ticker, "cash_flow_statement")

In [5]:
balance = form_statement_for_all_available_years(ticker, "balance_sheet")
income = form_statement_for_all_available_years(ticker, "income_statement")
cash = form_statement_for_all_available_years(ticker, "cash_flow_statement")
save_dataframe_to_csv(balance, "statements", "temp", "balance_sheet", "annual")
save_dataframe_to_csv(income, "statements", "temp", "income_statement", "annual")
save_dataframe_to_csv(cash, "statements", "temp", "cash_flow", "annual")

In [6]:
balance = pd.read_csv("statements/temp/balance_sheet_annual.csv", index_col=0)
income = pd.read_csv("statements/temp/income_statement_annual.csv", index_col=0)
cash = pd.read_csv("statements/temp/cash_flow_annual.csv", index_col=0)

In [7]:
balance_sheet = balance
income_statement = income
cash_flow_statement = cash

In [8]:
statement = income_statement
statement_mapping = income_statement_mapping.income_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31
INCOME STATEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Revenue,59283.00,48704.00,41518.00,39121.00,42294.00,40122.00,39807.00,NaN,NaN,NaN,NaN
OPERATING EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Revenue,17411.00,13626.00,13618.00,12016.00,13509.00,12912.00,14030.00,NaN,NaN,NaN,NaN
SG&A,10042.00,9634.00,8955.00,9455.00,10102.00,10074.00,10017.00,10313.00,11606.00,11911.00,12776.00
Research and Development,13548.00,12245.00,13397.00,9724.00,9752.00,10339.00,10261.00,6704.00,7180.00,7503.00,8168.00
Restucturing and Other Special Charges,337.00,661.00,575.00,626.00,632.00,776.00,651.00,619.00,1013.00,1709.00,664.00
Other Expense Non-Operating,1501.00,-1341.00,-890.00,129.00,-402.00,-500.00,189.00,1527.00,-11613.00,411.00,1116.00
mrk_CostsExpensesAndOther,42839.00,34825.00,35655.00,31950.00,33593.00,33601.00,35148.00,34097.00,24954.00,38488.00,38528.00
EBT,16444.00,13879.00,5863.00,7171.00,8701.00,6521.00,4659.00,5401.00,17283.00,5545.00,8739.00


In [9]:
statement = balance_sheet
statement_mapping = balance_sheet_mapping.balance_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31
CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash,12694.0,8096.0,8050.0,9676.0,7965.0,6092.0,6515.0,8524.0,7441.0,15621.0
Marketable Securities,498.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accounts Receivable,9450.0,9230.0,6803.0,6778.0,7071.0,6873.0,7018.0,6484.0,6626.0,7184.0
Inventory,5911.0,5953.0,5554.0,5978.0,5440.0,5096.0,4866.0,4700.0,5571.0,6226.0
Other Current Assets,7169.0,6987.0,4674.0,4277.0,4500.0,4299.0,4389.0,5140.0,4689.0,NaN
Total Current Assets,35722.0,30266.0,27764.0,27483.0,25875.0,24766.0,30614.0,29751.0,32605.0,35685.0
Long Term Marketable Securities,1015.0,370.0,785.0,1469.0,6233.0,12125.0,11416.0,13039.0,13515.0,9770.0
us-gaap_PropertyPlantAndEquipmentGrossAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_Land,295.0,326.0,336.0,343.0,333.0,365.0,412.0,490.0,541.0,550.0


In [10]:
statement = cash_flow_statement
statement_mapping = cash_flow_mapping.cash_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31
OPERATING ACTIVITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_IncomeLossFromContinuingOperationsIncludingPortionAttributableToNoncontrollingInterest,14526.0,12358.0,4523.0,5606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADJUSTMENTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_AdjustmentForAmortization,2085.0,1636.0,1817.0,1695.0,3103.0,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,1824.0,1578.0,1669.0,1615.0,1416.0,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_GoodwillAndIntangibleAssetImpairment,1749.0,302.0,1718.0,1040.0,296.0,646.0,NaN,NaN,NaN,NaN,NaN
Fair Value Adjustments for Non-Marketable Securities,1419.0,-1940.0,-1338.0,-170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff,0.0,1556.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mrk_ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOffAssetAcquisitionTwo,0.0,0.0,2660.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Deferred Income Taxes,-1568.0,187.0,-566.0,-560.0,-509.0,-2621.0,-1521.0,-764.0,-2600.0,-330.0,669.0
